In [ ]:
'''
###AWS 주피터  파이썬 버전 불일치 오류 => 가상환경에서 파이썬 버전을 6버전으로 ...############################################
conda create --name spark_tutorial python=3.6.8 pyspark jupyter
conda activate spark_tutorial
jupyter-notebook --ip=0.0.0.0 --no-browser --port=8889
'''

In [2]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Test").config("spark.executor.instances", "1").config("spark.executor.cores", "1").config("spark.executor.memory", "2g").config("spark.driver.memory", "2g").getOrCreate()



In [2]:
# Data Frame 3개 선언하기 
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [3]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [4]:
#내부조인
joinExpression = person["graduate_program"] == graduateProgram['id']
person.join(graduateProgram, joinExpression).show()

joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+--

In [5]:
## 외부조인
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [6]:
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()


+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



In [7]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()


+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [8]:
gradProgram2 = graduateProgram.union(spark.createDataFrame([
    (0, "Masters", "Duplicated Row", "Duplicated School")]))

In [9]:
gradProgram2.createOrReplaceTempView("gradProgram2")
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [10]:
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [11]:
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [18]:
## 9.2.2 CSV 파일 읽기 
## csv파일을 생성 및 여러가지 조건을 주어 불러올 수 있다. 
csvFile = spark.read.format("csv").option("header", "true").option("mode", "FAILFAST").option("inferSchema", "true").load("/home/lab13/data/2015-summary.csv")

csvFile.write.format("csv").mode("overwrite").option("sep", "\t").save("/home/lab13/data/my-tsv-file.tsv")

In [20]:
## 9.4 파케이 파일 
## 외부 파케이 파일 데이터를 읽어와보자 
spark.read.format("parquet")\
.load("/home/lab13/data/parquets/2010-summary.parquet").show(5)


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [23]:
## 파케이 파일 작성하기 
csvFile.write.format("parquet").mode("overwrite")\
  .save("/home/lab13/data/parquets/my-parquet-file.parquet")

In [ ]:
'''
Table : RDB에서 데이터를 저장하는 가장 기본적인 구조 
table = row + column 
table의 종류에는 (heap, partition, iot(index oriented table), cluster ...)

View : 논리적 table (물리적 table에 대한 window)
       물리적 table에 대한 window 
       보안(Data Access제안)
       SQL을 간결하게 사용하지 위해 
       
index : 검색 성능을 향상시키기 위함 
        -hashing 
        -b tree 
        (https://asfirstalways.tistory.com/339)
        (https://middleware.tistory.com/entry/%EC%9D%B8%EB%8D%B1%EC%8A%A4Index%EC%99%80-%ED%95%B4%EC%8B%B1Hashing)
        
'''


In [5]:
# C:\app\student\product\11.2.0\dbhome_1\jdbc\lib\ojdbc6.jar
pyspark --jars "/home/tutor/data/jars/ojdbc6.jar" <- 이거그냥 경로 아냐??파일질라에 넣으라는

query = "(select empno,ename,dname from emp, dept where emp.deptno = dept.deptno) emp"
empDF = spark.read.format("jdbc").option("url", "jdbc:oracle:thin:scott/oracle@//70.12.116.160:1521/orcl").option("dbtable", query).option("user", "scott").option("password", "oracle").option("driver", "oracle.jdbc.driver.OracleDriver").load()
empDF.printSchema()
empDF.show()

SyntaxError: invalid syntax (<ipython-input-5-8b7e5d71ca67>, line 2)

In [ ]:
#Spark home/conf 폴더의 spark-defaults.conf의 spark.driver.extraClassPath에 해당 jar 파일 경로를 추가 합니다.
spark.driver.extraClassPath  /home/lab13/data/jars/ojdbc6.jar

In [ ]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName("Python Spark SQL data source example").getOrCreate() 

query = "(select empno,ename,dname from emp, dept where emp.deptno = dept.deptno) emp"

df = spark.read.format("jdbc").option("url", "jdbc:oracle:thin:@127.0.01:1521/orcl").option("dbtable", query) .option("user", "scott").option("password", "oracle").load()
 df.show() 
spark.stop()


In [9]:
spark.read.text("/home/lab13/data/2010-12-01.csv").selectExpr("split(value, ',') as rows").show()

+--------------------+
|                rows|
+--------------------+
|[InvoiceNo, Stock...|
|[536365, 85123A, ...|
|[536365, 71053, W...|
|[536365, 84406B, ...|
|[536365, 84029G, ...|
|[536365, 84029E, ...|
|[536365, 22752, S...|
|[536365, 21730, G...|
|[536366, 22633, H...|
|[536366, 22632, H...|
|[536367, 84879, A...|
|[536367, 22745, P...|
|[536367, 22748, P...|
|[536367, 22749, F...|
|[536367, 22310, I...|
|[536367, 84969, B...|
|[536367, 22623, B...|
|[536367, 22622, B...|
|[536367, 21754, H...|
|[536367, 21755, L...|
+--------------------+
only showing top 20 rows



In [ ]:
'''pyspark, sparkR에 이어서 spark-sql 생성하기

먼저putty login 
cd ~
/home/lab13
ls -al    #여기서 .bashrc확인 
vi .bashrc
source .bashrc #한번 실행
spark-sql #입력 이후 생성자가 바뀌었는지 확인 

'''